<img width="8%" alt="Google Docs.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Docs.png" style="border-radius: 15%">

# Google Docs - Replace Placeholders in Document
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Google+Docs+-+Replace+Placeholders+in+Document:+Error+short+description">Bug report</a>

**Tags:** #googledocs #placeholderreplacement #productivity #googledocsmagic

**Author:** [Chris Arnold](https://chrisarnold.live/me/linkedin)

**Last update:** 2023-11-02 (Created: 2023-10-12)

**Description:** This notebook demonstrates how to replace placeholders in a Google Docs document, allowing you to quickly create personalized content from templates.

**References:**
- [Google Docs - API](https://developers.google.com/docs/api/how-tos/overview)
- [Google Workspace - Develop on Google Workspace](https://developers.google.com/workspace/guides/get-started)

## Input

### Import libraries

In [ ]:
try:
    from apiclient.discovery import build
    from google.oauth2.service_account import Credentials
except ModuleNotFoundError:
    !pip install google-api-python-client
    from apiclient.discovery import build
    from google.oauth2.service_account import Credentials

### Setup variables
**Pre-requisites** 
- You need to set up service account credentials to use Google Docs API. For detailed instructions on how to do this, please refer to the [Google Docs Python Quickstart guide](https://developers.google.com/docs/api/quickstart/python).
- Please also ensure you have the necessary permissions to access and modify the Google Docs document. If global editor access to the document is not preferable, the document needs to be shared with the service account email (xxx@docs-temps.iam.gserviceaccount.com).
- It is recommended that the placeholders in the documents are in the format `{{placeholder}}` (with double curly braces) to help with visibility and to avoid words that are not being placeholders from being replaced.

**Mandatory** 
- `service_account_file`: JSON file containing credentials related to service account
- `document_url` : URL of Google Doc file.
- `placeholder`: The placeholder text.
- `replacement_value`: The text to replace the placeholder with.

In [ ]:
service_account_file = "credentials.json"
document_url = "https://docs.google.com/document/d/1gUAb3J8YED90wZVxV6j0zI18hRh4I2y9JR7BqraJ-XY/edit"
placeholder = "NAME"
replacement_value = "Florent"

## Model

### Connect to GCP

This function will create a Docs API client using the service account credentials

In [ ]:
def gcp_connect(file_path):
    scope = "https://www.googleapis.com/auth/documents"
    credentials = Credentials.from_service_account_file(file_path, scopes=[scope])
    service = build('docs', 'v1', credentials=credentials) 
    return service

service = gcp_connect(service_account_file)

### Replace placeholder in the document

In [ ]:
def placeholder_replacement(
    service,
    document_url,
    placeholder,
    replacement_value
):
    """
    Replace a placeholder in a Google Docs document with a replacement value.

    Args:
        service (obj): Google Docs service.
        document_url (str): The URL of the Google Docs document.
        placeholder (str): The placeholder text.
        replacement_value (str): The text to replace the placeholder with.

    Returns:
        str: A message indicating the number of template occurrences replaced.
    """
    document_id = document_url.split("/d/")[-1].split("/")[0]
    
    request = {
        'replaceAllText': {
            'containsText': {
                'text': placeholder,
                'matchCase': True
            },
            'replaceText': replacement_value
        }
    }

    body = {
        'requests': [request]
    }
    response = service.documents().batchUpdate(documentId=document_id, body=body).execute()
    
    occurrences_changed = response.get('replies', [{}])[0].get('replaceAllText', {}).get('occurrencesChanged', 0)

    return f"{occurrences_changed} occurrences of the placeholder '{placeholder}' have been successfully replaced!"

placeholder_replacement(
    service,
    document_url,
    placeholder,
    replacement_value
)

## Output

### Display result

In [ ]:
print("To see the result, go to your Google Docs:", document_url)